In [6]:
# ANNOTATION CONVERSION

def yoloconversion(file, label, imagesize):
    dataframe = pd.read_csv(file, sep=" ", header=None)
    dataframe = dataframe.loc[dataframe[0] == label]

    xmin = (dataframe[1] - dataframe[3]/2) * imagesize[0]
    dataframe['xmin'] = xmin
    xmax = (dataframe[1] + dataframe[3]/2) * imagesize[0]
    dataframe['xmax'] = xmax
    ymin = (dataframe[2] - dataframe[4]/2) * imagesize[1]
    dataframe['ymin'] = ymin
    ymax = (dataframe[2] + dataframe[4]/2) * imagesize[1]
    dataframe['ymax'] = ymax
    
    if dataframe.shape[0] == 0:
        dataframe = [[0, 0, 0, 0, 0, 0, 0, 0, 0]]
    else:
        dataframe = dataframe.to_numpy()
        dataframe.astype(int)

    return dataframe # [x, y, w, h, class, xmin, xmax, ymin, ymax]

In [7]:
# RUN THE MODEL

def run(model, file, label, img_dim): # (model, img file, label to analize, img_dimensions[height, width])
    s = int(math.sqrt(img_dim[0] * img_dim[1])/2)
    results = model(file, size=s)
    dataframe = results.pandas().xyxy[0]
    dataframe = dataframe.loc[dataframe['class'] == label]

    if dataframe.shape[0] == 0:
        dataframe = [[img_dim[1]/2, img_dim[0]/2, img_dim[1]/2, img_dim[0]/2, 0, label, '']]
    else:
        dataframe = dataframe.to_numpy()

    return dataframe # [xmin, ymin, xmax, ymax, confidence, class, class_name]

In [8]:
# INTERSECTION OVER UNION

def intersectionoverunion(b1, b2): # [[xmin, ymin, xmax, ymax], [xmin, ymin, xmax, ymax]]
    dx = min(b1[2], b2[2]) - max(b1[0], b2[0])
    dy = min(b1[3], b2[3]) - max(b1[1], b2[1])

    try:
        iou = dx*dy / ((b1[2]-b1[0])*(b1[3]-b1[1]) + (b2[2]-b2[0])*(b2[3]-b2[1]) - dx*dy)
    except ZeroDivisionError:
        iou = 0
    return iou

In [9]:
import cv2
import cvzone
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
import math
import time

model = torch.hub.load('', 'custom', path='traffic_signs.pt', source='local')
model.iou = 0.6

labels = [0, 1, 2, 3]
imagesize = [1360, 800]

grid = [[4, 8, 12], [1, 4, 9]] # grid and factor
path_attack = 'C:/Users/cibar/Documents/MEGAsync/Programacion/Object detection/yolov5/traffic_signs/train/attack/' # output path
patch_color = [(0, 0, 255), (255, 0, 0), (0, 0, 0), (0, 255, 255)]

path_labels = 'traffic_signs/train/labels/' # input path
path_images = 'traffic_signs/train/images/' # input path

###################################################################################################################################

labels_files = os.listdir(path_labels)
images_files = os.listdir(path_images)
images_files_ne = [x.split('.')[0] for x in images_files] # images files without extension

Fusing layers... 
Model Summary: 224 layers, 7062001 parameters, 0 gradients
Adding AutoShape... 
YOLOv5  2021-6-13 torch 1.8.2+cpu CPU



In [ ]:
f = open('attack_output.txt', 'a')
total = len(images_files) * len(patch_color)
starttime = time.time()

for g in range(len(patch_color)):
    opacy = []
    for i in range(len(images_files)): # i: image number...
        current = g * len(images_files)+(i+1)
        f.write('IMAGE: ' + str(images_files[i]) + ' ==> ' + str(current) +'/'+ str(total) +'\n')
        clear_output()
        print('Completed: ' + str(round(100*current/total, 2)) + '%')
        print('ETA: ' + str(round(((time.time()-starttime)/(current/total))/60,0)) + ' min')

        img = cv2.imread(path_images+images_files[i]) # Image i to be analyzed
        blank_img = cv2.imread(path_images+images_files[i]) # A background for the transparent patch
        blank_img[:, :, :] = 0

        for h in range(len(labels)): # h: label number...
            f.write('> Label: ' + str(labels[h]) + '\n')
            obj_df = yoloconversion(path_labels+labels_files[i], h, imagesize) # [x, y, w, h, class, xmin, xmax, ymin, ymax]
            run_df = run(model, path_images+images_files[i], labels[h], imagesize) # [xmin, ymin, xmax, ymax, confidence, class, class_name]

            iou = [] # List of iou values for each object in the image
            for j in range(len(obj_df)): # j: object number

                if sum(obj_df[j]) == sum([0, 0, 0, 0, 0, 0, 0, 0, 0]): # If there is not an object with the laber 'h'
                    f.write('>>> Object ' + str(j) + ': There is no label ' + str(labels[h]) + ' in the image' +'\n')
                
                else:
                    iou_bb = [] # List of iou in each object (in case the model detect several bounding boxes in one image)
                    for k in range(len(run_df)): # k: bb number (per object)
                        iou_bb.append(intersectionoverunion([obj_df[j][5], obj_df[j][7], obj_df[j][6], obj_df[j][8]], run_df[k]))
                    iou = max(iou_bb)

                    xs = [(obj_df[j][6]-obj_df[j][5])*2/grid[0][0], (obj_df[j][8]-obj_df[j][7])*2/grid[0][0]] # extrasize according the 1st grid [xsx, xsy]
                    bb_xs = [int(obj_df[j][5]-xs[0]), int(obj_df[j][7]-xs[1]), int(obj_df[j][6]+xs[0]), int(obj_df[j][8]+xs[1])] # extended bb [xmin, ymin, xmax, ymax]
                    
                    # Limit the borders of the expanded bounding box
                    if bb_xs[0] < 0:
                        bb_xs[0] = 0
                    elif bb_xs[1] < 0:
                        bb_xs[1] = 0
                    elif bb_xs[2] > imagesize[0]:
                        bb_xs[2] = imagesize[0]
                    elif bb_xs[3] > imagesize[1]:
                        bb_xs[3] = imagesize[1]
                    
                    # Crop an image and create a blank image
                    cropped_img = img[bb_xs[1]:bb_xs[3], bb_xs[0]:bb_xs[2]] # cropped image [ymin:ymax, xmin:xmax]
                    run_df_cropped = run(model, cropped_img, labels[h], cropped_img.shape) # [xmin, ymin, xmax, ymax, confidence, class, class_name]
                    patch_img = np.zeros((cropped_img.shape[0], cropped_img.shape[1], cropped_img.shape[2]+1)) # image height*width and 3+1 channels
                    start_loc = [xs[0], xs[1]] # center location of the first patch

                    for l in range(len(grid[0])): # l: grid division number ==> [0, 1, 2, 3]
                        grid_size = [] # grid size [x, y]
                        grid_size.append(xs[0]*grid[0][0]/grid[0][l]/2)
                        grid_size.append(xs[1]*grid[0][0]/grid[0][l]/2)

                        for n in range(grid[0][l]+1): # n: y coordinate of the patch
                            for o in range(grid[0][l]+1): # o: x coordinate of the patch
                                loc = [start_loc[0] + o*grid_size[0], start_loc[1] + n*grid_size[1]]
                                patch_img_single = cropped_img.copy()

                                minimo = [int(loc[0]-grid_size[0]), int(loc[1]-grid_size[1])]
                                maximo = [int(loc[0]+grid_size[0]), int(loc[1]+grid_size[1])]

                                cv2.rectangle(patch_img_single, (int(loc[0]-grid_size[0]), int(loc[1]-grid_size[1])), (int(loc[0]+grid_size[0]), int(loc[1]+grid_size[1])), patch_color[g], -1)
                                run_df_patch_single = run(model, patch_img_single, labels[h], cropped_img.shape)
                                iou_patch_single = intersectionoverunion([xs[0], xs[1], xs[0]+(obj_df[j][6]-obj_df[j][5]), xs[1]+(obj_df[j][8]-obj_df[j][7])], run_df_patch_single[0])

                                if iou_patch_single < 0:
                                    iou_dif = 0
                                elif iou_patch_single > iou:
                                    iou_dif = 0
                                else:
                                    iou_dif = iou - iou_patch_single

                                # Draw over the blank image
                                patch_img_single = np.zeros((cropped_img.shape[0], cropped_img.shape[1], cropped_img.shape[2]+1))
                                patch_img_single[int(loc[0]-grid_size[0]):int(loc[0]+grid_size[0]), int(loc[1]-grid_size[1]):int(loc[1]+grid_size[1]), 3] = iou_dif # [xmin:xmax, xmin:ymax, channel] A patch img with a opacy of iou_difference value
                                patch_img = patch_img + patch_img_single*grid[1][l] # Add the opacy value to the blank image, all patches

####################################################################################################################################
                    # Calculate the alpha
                    if np.amax(patch_img) == 0: # Skip the false positives (the 0 dimension patch never affects the precision)
                        f.write('>>> Object ' + str(j) + ': False positive' +'\n')
                    else:
                        alpha = 0
                        iou_alpha = 1
                        combo = 0
                        numero = 1
                        while numero > 0:
                            alpha = alpha + 1

                            patch_img_alpha = alpha/np.amax(patch_img) * patch_img
                            patch_img_alpha[:, :, 0] = patch_color[g][0] # set the Blue patch color
                            patch_img_alpha[:, :, 1] = patch_color[g][1] # set the Green patch color
                            patch_img_alpha[:, :, 2] = patch_color[g][2] # set the Red patch color

                            cv2.imwrite('patch_img_alpha.png', patch_img_alpha) # Save the transparent patch
                            patch_img_alpha = cv2.imread('patch_img_alpha.png', cv2.IMREAD_UNCHANGED) # Load the transparent patch unchanged

                            img_cropped_patch = cvzone.overlayPNG(cropped_img, patch_img_alpha)
                            run_alpha = run(model, img_cropped_patch, h, img_cropped_patch.shape)
                            iou_alpha = intersectionoverunion([xs[0], xs[1], xs[0]+(obj_df[j][6]-obj_df[j][5]), xs[1]+(obj_df[j][8]-obj_df[j][7])], run_alpha[0])

                            if iou_alpha == 0:
                                combo = combo + 1
                            else:
                                combo = 0
                            if combo > 15:
                                numero = 0
                            if alpha > 254:
                                numero = 0

                        f.write('>>> Object ' + str(j) + ': Ok --> ' + str(alpha) + '\n')
                        opacy.append(alpha)

####################################################################################################################################
#                        # Plot a IOU - alpha graph
#                        alpha_iou = [[], []]
#                        alpha = 0
#                        while alpha < 256:
#                            patch_img_alpha = alpha/np.amax(patch_img) * patch_img
#                            patch_img_alpha[:, :, 0] = patch_color[g][0] # set the Blue patch color
#                            patch_img_alpha[:, :, 1] = patch_color[g][1] # set the Green patch color
#                            patch_img_alpha[:, :, 2] = patch_color[g][2] # set the Red patch color
#
#                            cv2.imwrite('patch_img_alpha.png', patch_img_alpha) # Save the transparent patch
#                            patch_img_alpha = cv2.imread('patch_img_alpha.png', cv2.IMREAD_UNCHANGED) # Load the transparent patch unchanged
#
#                            img_cropped_patch = cvzone.overlayPNG(cropped_img, patch_img_alpha)
#
#                            run_alpha = run(model, img_cropped_patch, h, img_cropped_patch.shape)
#                            iou_alpha = intersectionoverunion([xs[0], xs[1], xs[0]+(obj_df[j][6]-obj_df[j][5]), xs[1]+(obj_df[j][8]-obj_df[j][7])], run_alpha[0])
#
#                            alpha_iou[0].append(alpha)
#                            alpha_iou[1].append(iou_alpha)
#                            alpha = alpha + 1
#
#                        plt.plot(alpha_iou[0], alpha_iou[1])
#                        plt.xlim(0, 255)
#                        plt.ylim(0, 1)
#                        plt.savefig('C:/Users/cibar/Documents/MEGAsync/Programacion/Object detection/yolov5/traffic_signs/train/attack/' + str(images_files_ne[i]) + "_alpha_iou_" + str(h) + "_" + str(j) + "_.png")
#                        plt.clf()
####################################################################################################################################

                        # Crop larges partches
                        pl = [int(obj_df[j][5] - xs[0]), int(obj_df[j][7] - xs[1])] # patch location
                        rec = [0, 0]
                        if pl[0] < 0:
                            pl[0] = 0
                            rec[0] = -pl[0]
                        elif pl[1] < 0:
                            pl[1] = 0
                            rec[1] = -pl[1]
                        patch_img = patch_img[rec[0]:, rec[1]:,] # Crop the image to avoid being out of the image

                        img = cvzone.overlayPNG(img, patch_img_alpha, pl) # Overlay in the location [x, y]
                        blank_img = cvzone.overlayPNG(blank_img, patch_img_alpha, pl)

        cv2.imwrite(path_attack + str(g) + images_files[i], img) # Location + color + file
        #cv2.imwrite(path_attack + str(g) + images_files_ne[i] + '_patch.png', blank_img) # only patch
        f.write('########################################################')

    f.write('Average alpha: ' + str(sum(opacy)/len(opacy)) +'\n')
    f.write('########################################################' +'\n')

In [ ]:
# INFERENCE

model = torch.hub.load('', 'custom', path='traffic_signs.pt', source='local')
model.iou = 0.7
imagesize = [1360, 800]
s = int(math.sqrt(imagesize[0] * imagesize[1])/2)


print(yoloconversion('traffic_signs/val/labels/00712.txt', 0, [1360, 800]))
print("-"*100)

image = 'traffic_signs/val/images/00712.jpg'
results = model(image, size=s)
print(results.pandas().xyxy[0])
print("-"*100)

image = 'traffic_signs/val/attack/00712.jpg'
results = model(image, size=s)
print(results.pandas().xyxy[0])

In [ ]:

import requests
import json

def get():
    endpoint = 'https://ipinfo.io/json'
    response = requests.get(endpoint, verify = True)

    if response.status_code != 200:
        return 'Status:', response.status_code, 'Problem with the request. Exiting.'
        exit()

    data = response.json()

    return data['ip']

#get my ip
my_ip = get()

#print my ip
print(type(my_ip))

In [12]:
!python train.py --img 640 --batch 32 --epochs 5 --data traffic_signs.yaml --weights yolov5s.pt --cache

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5


Traceback (most recent call last):
  File "train.py", line 512, in <module>
    opt.data, opt.cfg, opt.hyp = check_file(opt.data), check_file(opt.cfg), check_file(opt.hyp)  # check files
  File "c:\Users\cibar\Documents\MEGAsync\Programacion\Object detection\yolov5\utils\general.py", line 219, in check_file
    assert len(files) == 1, f"Multiple files match '{file}', specify exact path: {files}"  # assert unique
AssertionError: Multiple files match 'traffic_signs.yaml', specify exact path: ['.\\data\\traffic_signs.yaml', '.\\traffic_signs\\traffic_signs.yaml']


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="EjbgFCkKPcbJLcZZSHd4")
project = rf.workspace().project("traffic-signs-kpv6r")
dataset = project.version(2).download("yolov5")

In [ ]:
!python train.py --img 640 --batch 8 --epochs 5 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache